<a href="https://colab.research.google.com/github/WebXpertAlando/Streamlit_Deployment_Loan_Defaulting/blob/main/Loan_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [ ]:

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Loan_default.csv")

# Data Overview

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.tail()

In [ ]:
df.duplicated().sum()

# Descriptive Statistics

In [ ]:
df.describe().T

# Visualization

**Default Distribution**

In [ ]:
# # Define function to group age into bins with specific ranges
# def group_age(df, age_column='Age'):
#     # Define the age bins and labels with specified intervals
#     bins = [18, 25, 30, 35, 40, 45, 50, 55, 60, 65, float('inf')]
#     labels = ['18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69']

#     # Create a new column 'Age Group' using pd.cut
#     df['Age Group'] = pd.cut(df[age_column], bins=bins, labels=labels, right=False)
#     return df

# # Apply the function to your DataFrame
# df = group_age(df)


In [ ]:
# # Create a count plot with 'Age Group' on the x-axis and 'Default' as hue
# plt.figure(figsize=(8, 5))
# sns.countplot(x='Age Group', data=df, hue='Default', palette='magma')

# # Set plot labels and title
# plt.xlabel('Age Group')
# plt.ylabel('Count')
# plt.title('Count Plot of Defaults by Age Group')
# plt.legend(title='Default Status', loc='upper right')

# 0 non defaulters 1 Defaulters


In [ ]:
df_num = df.select_dtypes(exclude = ['object']).drop(['Default','DTIRatio','LoanTerm'], axis = 1)
df_num

In [ ]:
df['NumCreditLines'].unique()

In [ ]:
for col in df_num:
    sns.boxplot(x = 'Default' , y = df[col], data =df , palette = 'dark')
    plt.show()

# Interpretation

1. Older people are seen less likely to default

2. The dafaulters are seen with lower avg income

3. The dafaulters are seen with larger Loan Amount and lesser credit score

4. People having higher Interest rate are more likely to default

# Correlation

In [ ]:
df_num.corr()

In [ ]:
numerical_cols = df.select_dtypes(include=['float64', 'int64'])
skewness = numerical_cols.skew()
skewness

In [ ]:
  plt.figure(figsize = (10,6))
sns.heatmap(df_num.corr(), annot = True, cmap = 'YlGnBu')

**No significant correlation found**

# **ScatterPlot**

In [ ]:
sns.scatterplot( x = 'Income' , y = 'LoanAmount', data =df,hue = 'Default')

No as such relationship found , the defaulters are spread between both low and high income

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import tree

# Feature and target segregation

In [ ]:
# dropping Loan ID
df.drop('LoanID', axis = 1 , inplace = True)

In [ ]:
df.head()

In [ ]:
x = df.drop('Default', axis = 1)
y = df['Default']

# Label Encoding

In [ ]:
# filtering only non numeric col
df_obj = df.select_dtypes(include=['object'])

In [ ]:
enc = LabelEncoder()
for col in df_obj:
    x[col] = enc.fit_transform(x[col])

Categorical(Textual) columns are now converted into Numerical data

In [ ]:
x.head()

# Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
x[x.columns] = scaler.fit_transform(x[x.columns])

In [ ]:
x.head()

**Features are now standardized**

# Splitting the Data into train and test part

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
x_train.shape, x_test.shape

# Samping technique - SMOTE


- For each sample in the minority class, SMOTE identifies its nearest neighbors (usually k-nearest neighbors).

- Generate synthetic samples: It then creates new synthetic samples by interpolating between the minority class sample and its neighbors. These synthetic samples are added to the dataset, improving the balance between classes.

In [ ]:
from imblearn.over_sampling import SMOT

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=1)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Apply SMOTE only to the training data
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

# Model Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

# Initialize models
models = {
    'Decision Tree': DecisionTreeClassifier(max_depth=4, random_state=1, criterion='entropy'),
    'Logistic Regression': LogisticRegression(random_state=1),
    'Random Forest': RandomForestClassifier(random_state=1),
    'Gradient Boosting': GradientBoostingClassifier(random_state=1),
    'XGBoost': xgb.XGBClassifier(random_state=1)
}

# List to store model metrics
models_metrics = []

# Evaluate each model
for name, model in models.items():
    # Fit the model on the resampled training set
    model.fit(x_train_smote, y_train_smote)

    # Predict on the test data (unbalanced original test set)
    y_pred = model.predict(x_test)
    y_pred_proba = model.predict_proba(x_test)[:, 1] if hasattr(model, 'predict_proba') else None

    # Calculate the metrics
    accuracy = round(accuracy_score(y_test, y_pred), 4)
    f1 = round(f1_score(y_test, y_pred), 4)
    precision = round(precision_score(y_test, y_pred), 4)
    recall = round(recall_score(y_test, y_pred), 4)
    auc = round(roc_auc_score(y_test, y_pred_proba), 4) if y_pred_proba is not None else 'N/A'

    # Append the metrics to the list
    models_metrics.append({
        "Model": name,
        "Accuracy": accuracy,
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall,
        "AUC": auc
    })

# Convert the results into a DataFrame for better visualization
metrics_df = pd.DataFrame(models_metrics)

# Print the results table
metrics_df


# Fitting the training Data

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming x_train, x_test, y_train, y_test are already split

# Apply SMOTE to the training data
smote = SMOTE(random_state=1)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

# Train the XGBoost model
import xgboost as xgb
xgb_model = xgb.XGBClassifier(random_state=1)
xgb_model.fit(x_train_smote, y_train_smote)

# Make predictions
y_pred = xgb_model.predict(x_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test,y_pred))

# Predictions

In [ ]:
y_pred = xgb_model.predict(x_test)

# Accuracy Evaluation

In [ ]:
accuracy_score(y_test,y_pred)

## Feature importance

In [ ]:
# Get feature importances
feature_importances = xgb_model.feature_importances_

# Create a DataFrame for better visualization (assuming X_train has the feature names)
features = x_train.columns
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')

# Add titles and labels
plt.title('Feature Importance from xgboost', fontsize=16)
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
importance_df.tail()

**Feature Importance Explanation**:
1.	Age (15.6%): Age is the most significant factor. Typically, older customers are more financially stable and less likely to default, while younger borrowers might face more uncertainty in their financial situations.
2.	Interest Rate (9.1%): Higher interest rates increase the monthly payment burden, making it harder for customers to keep up with payments, thus raising the risk of default.
3.	Has Dependents (8.9%): Customers with dependents often have higher financial obligations, which can increase the strain on their resources, making them more susceptible to defaulting on loans.
4.	Has Co-Signer (8.0%): Having a co-signer reduces the risk of default because the co-signer provides an additional layer of financial responsibility. Borrowers with a co-signer are considered less risky.
5.	Income (7.6%): While income is important, it ranks lower than other factors. Higher income generally means more disposable income to manage loan payments, though it's not the only determinant of financial health.
6.	Loan Amount (6.7%): Larger loan amounts are a financial burden, and a higher loan size relative to income could increase default risk.

Additional Factors:

•	Months Employed (6.7%): Longer employment history suggests more stable income, reducing default risk.


**Recommendation:**
1.	Focus on Age and Dependents: When evaluating loan applications, consider age and whether the customer has dependents as significant risk factors. Younger borrowers and those with dependents may require more stringent loan conditions or additional screening.
3.	Promote Co-Signer Options: Encourage customers, especially younger or higher-risk borrowers, to secure a co-signer. This not only lowers their default risk but also makes loans more accessible.
3.	Credit Score Monitoring: Ensure that customers with lower credit scores undergo a stricter approval process. Offer credit improvement plans for borderline cases to help them qualify for better loan terms.



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])  # Specify the labels (0, 1) or use your specific classes
disp.plot()

# Show the plot
plt.title("Confusion Matrix")
plt.show()

**So, we are getting approximately 89% accuracy**